In [1]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='your module name')

In [2]:
df = pd.read_csv('data.csv',index_col=0)
index = df.index
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=df.seg_text)
                        .toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),
                                     counter.vocabulary_.keys())))
bow_docs.columns = terms
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize,)
tfidf_docs =tfidf.fit_transform(raw_documents=df.text).toarray()
tfidf_docs = tfidf_docs - tfidf_docs.mean(axis=0)


try:
    print('rumor0:'+df.loc['rumor0'].text)
    print(bow_docs.loc['rumor0'][bow_docs.loc['rumor0'] > 0].head())
except KeyError:
    print('rumor0!:'+df.loc['rumor0!'].text)
    print(bow_docs.loc['rumor0!'][bow_docs.loc['rumor0!'] > 0].head())

rumor0!:脑出血不能引起病人剧烈的头痛。
不能    1
剧烈    1
头痛    1
引起    1
病人    1
Name: rumor0!, dtype: int64


In [3]:
sentiments = df.sentiments
sentiment_dict = np.load('sentiment_dict.npy',allow_pickle=True).item()

In [4]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia100 = LDiA(n_components=100, learning_method='batch')
ldia100 = ldia100.fit(bow_docs)
ldia_topic_vectors_100 = ldia100.transform(bow_docs)
columns100 = ['topic{}'.format(i) for i in range(ldia100.components_.shape[0])]
ldia_topic_vectors_100_doc = pd.DataFrame(ldia_topic_vectors_100,index=index, columns=columns100)
ldia_topic_vectors_100_sentiment = np.array([ldia_topic_vectors_100[i]*sentiments[i] for i in range(len(sentiments))])
ldia_topic_vectors_100_sentiment_doc = pd.DataFrame(ldia_topic_vectors_100_sentiment,index=index, columns=columns100)

ldia300 = LDiA(n_components=300, learning_method='batch')
ldia300 = ldia100.fit(bow_docs)
ldia_topic_vectors_300 = ldia300.transform(bow_docs)
columns300 = ['topic{}'.format(i) for i in range(ldia300.components_.shape[0])]
ldia_topic_vectors_300_doc = pd.DataFrame(ldia_topic_vectors_300,index=index, columns=columns300)
ldia_topic_vectors_300_sentiment = np.array([ldia_topic_vectors_300[i]*sentiments[i] for i in range(len(sentiments))])
ldia_topic_vectors_300_sentiment_doc = pd.DataFrame(ldia_topic_vectors_300_sentiment,index=index, columns=columns300)

NameError: name 'ldia100_topic_vectors_sentiment' is not defined

In [ ]:
from sklearn.decomposition import PCA
k=min(len(tfidf.vocabulary_),len(df))
k=100
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)
nums = pca.singular_values_.tolist()
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors_100 = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors_100_doc = pd.DataFrame(pca_topic_vectors_100, columns=columns, index=index)
pca_topic_vectors_100_sentiment = np.array([pca_topic_vectors_100[i]*sentiments[i] for i in range(len(sentiments))])
pca_topic_vectors_100_sentiment_doc = pd.DataFrame(pca_topic_vectors_100_sentiment, columns=columns, index=index)
k=300
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)
nums = pca.singular_values_.tolist()
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors_300 = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors_300_doc = pd.DataFrame(pca_topic_vectors_300, columns=columns, index=index)
pca_topic_vectors_300_sentiment = np.array([pca_topic_vectors_300[i]*sentiments[i] for i in range(len(sentiments))])
pca_topic_vectors_300_sentiment_doc = pd.DataFrame(pca_topic_vectors_300_sentiment, columns=columns, index=index)

In [ ]:
tfidf.vocabulary_
#根据词项的频率对词汇表进行排序
#当对某个不按照最左边元素排序的序列解压并在排序后重新压缩时，可以使用zip(*sorted(zip(...)))
column_nums , terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))
weights = pd.DataFrame(pca.components_, columns=terms,
                       index = ['topic{}'.format(i) for i in range(pca.components_.shape[0])])
pd.options.display.max_columns = 8

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100,n_iter=100)
svd_topic_vectors_100 = svd.fit_transform(tfidf_docs)
svd_topic_vectors_100 = (svd_topic_vectors_100.T / np.linalg.norm(svd_topic_vectors_100,axis=1)).T
svd_topic_vectors_100_doc = pd.DataFrame(svd_topic_vectors_100, columns=columns[:100], index=index)
svd_topic_vectors_100_sentiment = np.array([svd_topic_vectors_100[i]*sentiments[i] for i in range(len(sentiments))])
svd_topic_vectors_100_sentiment_doc = pd.DataFrame(svd_topic_vectors_100_sentiment, columns=columns[:100], index=index)

svd = TruncatedSVD(n_components=300,n_iter=100)
svd_topic_vectors_300 = svd.fit_transform(tfidf_docs)
svd_topic_vectors_300 = (svd_topic_vectors_300.T / np.linalg.norm(svd_topic_vectors_300,axis=1)).T
svd_topic_vectors_300_doc = pd.DataFrame(svd_topic_vectors_300, columns=columns[:300], index=index)
svd_topic_vectors_300_sentiment = np.array([svd_topic_vectors_300[i]*sentiments[i] for i in range(len(sentiments))])
svd_topic_vectors_300_sentiment_doc = pd.DataFrame(svd_topic_vectors_300_sentiment, columns=columns[:300], index=index)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split

def lda_predic (name,vector,df,over_sampling=False):
    X_train,X_test,y_train,y_test = train_test_split(vector,df.rumor,test_size=0.2 ,random_state=42)
    if over_sampling:
        smote = SMOTE(random_state=42)
        # 进行过采样
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
        print("过采样前训练集中类别0的数量：", sum(y_train==0))
        print("过采样前训练集中类别1的数量：", sum(y_train==1))

        print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
        print("过采样后训练集中类别1的数量：", sum(y_train_res==1))

    lda = LDA(n_components=1)
    lda = lda.fit(X_train_res, y_train_res)
    df[name] = lda.predict(vector)
    round(float(lda.score(X_test, y_test)), 3)
    from sklearn import metrics
    y_true = df.loc[y_test.index].rumor
    y_pred = df.loc[y_test.index][name]
    loss_total = 0
    acc = metrics.accuracy_score(y_true, y_pred)
    report = metrics.classification_report(y_true, y_pred, target_names=["non-rumor","rumor"], digits=4)
    confusion = metrics.confusion_matrix(y_true, y_pred)
    print(report)
    with open("report/"+name+'.txt', 'w', encoding='utf-8') as f:
        print(report,file=f)
    print(confusion)
    with open("report_matrix/"+name+'_matrix'+'.txt', 'w', encoding='utf-8') as f:
        print(confusion,file=f)
    np.savez("report_matrix/"+name+'_matrix'+'.npz', matrix= confusion)

In [ ]:
lda_predic('tfidf',tfidf_docs,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502


In [ ]:
lda_predic('pca_100',pca_topic_vectors_100_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9564    0.7127    0.8168       369
       rumor     0.6241    0.9362    0.7489       188

    accuracy                         0.7882       557
   macro avg     0.7902    0.8245    0.7829       557
weighted avg     0.8442    0.7882    0.7939       557

[[263 106]
 [ 12 176]]


In [ ]:
lda_predic('pca_300',pca_topic_vectors_300_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9164    0.7425    0.8204       369
       rumor     0.6318    0.8670    0.7309       188

    accuracy                         0.7846       557
   macro avg     0.7741    0.8048    0.7757       557
weighted avg     0.8203    0.7846    0.7902       557

[[274  95]
 [ 25 163]]


In [ ]:
lda_predic('pca_100_sentiment',pca_topic_vectors_100_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.8918    0.7371    0.8071       369
       rumor     0.6151    0.8245    0.7045       188

    accuracy                         0.7666       557
   macro avg     0.7534    0.7808    0.7558       557
weighted avg     0.7984    0.7666    0.7725       557

[[272  97]
 [ 33 155]]


In [ ]:
lda_predic('pca_300_sentiment',pca_topic_vectors_300_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.8797    0.7534    0.8117       369
       rumor     0.6224    0.7979    0.6993       188

    accuracy                         0.7684       557
   macro avg     0.7511    0.7756    0.7555       557
weighted avg     0.7929    0.7684    0.7737       557

[[278  91]
 [ 38 150]]


In [ ]:
lda_predic('svd_100',svd_topic_vectors_100_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.8780    0.6829    0.7683       369
       rumor     0.5667    0.8138    0.6681       188

    accuracy                         0.7271       557
   macro avg     0.7224    0.7484    0.7182       557
weighted avg     0.7730    0.7271    0.7345       557

[[252 117]
 [ 35 153]]


In [ ]:
lda_predic('svd_300',svd_topic_vectors_300_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9357    0.7100    0.8074       369
       rumor     0.6137    0.9043    0.7312       188

    accuracy                         0.7756       557
   macro avg     0.7747    0.8071    0.7693       557
weighted avg     0.8270    0.7756    0.7817       557

[[262 107]
 [ 18 170]]


In [ ]:
lda_predic('svd_100_sentiment',svd_topic_vectors_100_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.8691    0.7019    0.7766       369
       rumor     0.5753    0.7926    0.6667       188

    accuracy                         0.7325       557
   macro avg     0.7222    0.7472    0.7216       557
weighted avg     0.7700    0.7325    0.7395       557

[[259 110]
 [ 39 149]]


In [ ]:
lda_predic('svd_300_sentiment',svd_topic_vectors_300_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9347    0.6206    0.7459       369
       rumor     0.5513    0.9149    0.6880       188

    accuracy                         0.7199       557
   macro avg     0.7430    0.7677    0.7170       557
weighted avg     0.8053    0.7199    0.7264       557

[[229 140]
 [ 16 172]]


In [ ]:
lda_predic('ldia_100',ldia_topic_vectors_100_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.7187    0.6369    0.6753       369
       rumor     0.4174    0.5106    0.4593       188

    accuracy                         0.5943       557
   macro avg     0.5680    0.5737    0.5673       557
weighted avg     0.6170    0.5943    0.6024       557

[[235 134]
 [ 92  96]]


In [ ]:
lda_predic('ldia_300',ldia_topic_vectors_300_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.7483    0.5962    0.6637       369
       rumor     0.4335    0.6064    0.5055       188

    accuracy                         0.5996       557
   macro avg     0.5909    0.6013    0.5846       557
weighted avg     0.6420    0.5996    0.6103       557

[[220 149]
 [ 74 114]]


In [ ]:
lda_predic('ldia_100_sentiment',ldia_topic_vectors_100_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.7391    0.5528    0.6326       369
       rumor     0.4128    0.6170    0.4947       188

    accuracy                         0.5745       557
   macro avg     0.5760    0.5849    0.5636       557
weighted avg     0.6290    0.5745    0.5860       557

[[204 165]
 [ 72 116]]


In [ ]:
lda_predic('ldia_300_sentiment',ldia_topic_vectors_300_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.7754    0.5989    0.6758       369
       rumor     0.4559    0.6596    0.5391       188

    accuracy                         0.6194       557
   macro avg     0.6157    0.6292    0.6075       557
weighted avg     0.6676    0.6194    0.6297       557

[[221 148]
 [ 64 124]]
